In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [2]:
db=pd.read_csv('Cleaned_Nepali_dataset.csv')
db.head()

,Text,Target
0,गुठी विधेक ल्याएर ठमेल राज गुठि जग्गा छाया सेन...,0
1,दले देश सकेछन बेचे खान सुरू गरेछन दले लखेटनु पछ ।,1
2,नेपाल ससकृती ध्वस्त पार्ने योजना !,1
3,मठ मन्दिर गुम्बा जग्गा हरु भुमाफिया नजर परे हु...,1
4,नेपाल कल कर्खाना नदि नाला बेची सके मठ मन्दीर ब...,1


In [3]:

db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2639 entries, 0 to 2638
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    2639 non-null   object
 1   Target  2639 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 41.4+ KB


In [4]:
db.describe()

,Target
count,2639.000000
mean,1.275862
std,1.379088
min,0.000000
25%,0.000000
50%,1.000000
75%,2.000000
max,5.000000


In [5]:
db.isnull().sum()

Text      0
Target    0
dtype: int64

# Preprocessing

In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [5]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\shres\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shres\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\shres\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
stop_word = nltk.corpus.stopwords.words('nepali')
print('Number of stop words', len(stop_word))

Number of stop words 255


In [ ]:
import re

def tokenize(text):
    if not isinstance(text, str):
        text = str(text)

    # Lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Strip whitespace
    text = text.strip()

    # Sentence tokenization
    sentences = nltk.sent_tokenize(text)

    # Word tokenization and remove stopwords
    stop_words = nltk.corpus.stopwords.words('nepali')
    words = [
        word for sentence in sentences
        for word in nltk.word_tokenize(sentence)
        if word.lower() not in stop_words
    ]

    return words


In [24]:

db['text_tokenize']=db['Text'].apply(tokenize)
db.head()

,Text,Target,text_tokenize,text_idf
0,गुठी विधेक ल्याएर ठमेल राज गुठि जग्गा छाया सेन...,0,"[गठ, वधक, लयएर, ठमल, रज, गठ, जगग, छय, सनटर, जस...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,दले देश सकेछन बेचे खान सुरू गरेछन दले लखेटनु पछ ।,1,"[दल, दश, सकछन, बच, खन, सर, गरछन, दल, लखटन, पछ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,नेपाल ससकृती ध्वस्त पार्ने योजना !,1,"[नपल, ससकत, धवसत, परन, यजन]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,मठ मन्दिर गुम्बा जग्गा हरु भुमाफिया नजर परे हु...,1,"[मठ, मनदर, गमब, जगग, हर, भमफय, नजर, पर, हन, वधक]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,नेपाल कल कर्खाना नदि नाला बेची सके मठ मन्दीर ब...,1,"[नपल, कल, करखन, नद, नल, बच, सक, मठ, मनदर, बच, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


# vectorization

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [26]:
db["text_tokenize"] = db["text_tokenize"].apply(lambda tokens: " ".join(tokens))

In [27]:
vectorizer = TfidfVectorizer()
text_idf = vectorizer.fit_transform(db["text_tokenize"])

In [28]:

db["text_idf"] = list(text_idf.toarray())
db.head(3)

,Text,Target,text_tokenize,text_idf
0,गुठी विधेक ल्याएर ठमेल राज गुठि जग्गा छाया सेन...,0,गठ वधक लयएर ठमल रज गठ जगग छय सनटर जसत जगउन लयउ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,दले देश सकेछन बेचे खान सुरू गरेछन दले लखेटनु पछ ।,1,दल दश सकछन बच खन सर गरछन दल लखटन पछ,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,नेपाल ससकृती ध्वस्त पार्ने योजना !,1,नपल ससकत धवसत परन यजन,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [22]:
idf_df = db[[ "text_idf", "Target"]].copy()

In [23]:
idf_df.head()

,text_idf,Target
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1


In [30]:
idf_df["text_idf"] = idf_df["text_idf"].apply(lambda x: np.array(x).flatten())

# Creating model for hate speech analysis

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
x= idf_df[[ "text_idf"]]
y= idf_df["Target"]

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)